In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# === 1. Load Dataset ===
df = pd.read_csv("students_adaptability_level_online_education.csv")
print(df.isnull().sum())
df = df.dropna()

# === 2. Encoding & Preprocessing ===
df['Device'] = df['Device'].map({'Mobile': 1, 'Tab': 2, 'Computer': 3})
df['Financial Condition'] = df['Financial Condition'].map({'Poor': 1, 'Mid': 2, 'Rich': 3})
df['Internet Quality'] = df['Internet Type'].map({'Mobile Data': 1, 'Wifi': 3})
df['Internet Quality'] = np.where(df['Network Type'] == '2G', 1,
                          np.where(df['Network Type'] == '3G', 2,
                          np.where(df['Network Type'] == '4G', 2,
                          df['Internet Quality'])))

# Encode target
le_flex = LabelEncoder()
df['Flexibility_Level_Label'] = le_flex.fit_transform(df['Flexibility Level'])

# === 3. Fuzzifikasi Functions ===
# Fungsi keanggotaan trapezoidal
def trapezoid(x, a, b, c, d):
    if x <= a or x >= d:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x <= c:
        return 1
    elif c < x < d:
        return (d - x) / (d - c)

# Fuzzifikasi usia (contoh: young, adult, mature)
def fuzzify_age(age):
    return {
        'young': trapezoid(age, 10, 10, 18, 24),
        'adult': trapezoid(age, 20, 25, 30, 35),
        'mature': trapezoid(age, 30, 35, 45, 60)
    }

# Fuzzifikasi device
def fuzzify_device(device):
    return {
        'low': trapezoid(device, 1, 1, 1.5, 2),
        'high': trapezoid(device, 2, 2.5, 3, 3)
    }

# Fuzzifikasi finansial
def fuzzify_financial(financial):
    return {
        'poor': trapezoid(financial, 1, 1, 1.5, 2),
        'rich': trapezoid(financial, 2, 2.5, 3, 3)
    }

# Fuzzifikasi kualitas internet
def fuzzify_internet(quality):
    return {
        'bad': trapezoid(quality, 1, 1, 1.5, 2),
        'good': trapezoid(quality, 2, 2.5, 3, 3)
    }


# === 4. Inference Mamdani ===
def mamdani_inference(age, device, financial, internet):
    a = fuzzify_age(age)
    d = fuzzify_device(device)
    f = fuzzify_financial(financial)
    i = fuzzify_internet(internet)

    rule_low = min(a['young'], d['low'], f['poor'], i['bad'])       # Rule 1
    rule_moderate = min(a['adult'], d['low'], f['rich'], i['good']) # Rule 2
    rule_high = min(a['mature'], d['high'], f['rich'], i['good'])   # Rule 3

    return {'low': rule_low, 'moderate': rule_moderate, 'high': rule_high}

# === 5. Defuzzifikasi Mamdani ===
def defuzzify_mamdani(output):
    scores = {'low': 0.2, 'moderate': 0.5, 'high': 0.8}
    numerator = sum(output[k] * scores[k] for k in output)
    denominator = sum(output.values())
    return numerator / denominator if denominator else 0

# === 6. Defuzzifikasi Sugeno ===
def defuzzify_sugeno(age, device, financial, internet):
    age_score = age / 40
    device_score = device / 3
    financial_score = financial / 3
    internet_score = internet / 3
    return 0.25 * (age_score + device_score + financial_score + internet_score)

# === 7. Skoring dan Klasifikasi ===
def predict_flexibility(score):
    if score < 0.33:
        return 1  # Low
    elif score < 0.66:
        return 2  # Moderate
    else:
        return 0  # High

# === 8. Prediksi Seluruh Data ===
mamdani_preds = []
sugeno_preds = []

for _, row in df.iterrows():
    # Mamdani
    fuzzy_out = mamdani_inference(row['Age'], row['Device'], row['Financial Condition'], row['Internet Quality'])
    mamdani_score = defuzzify_mamdani(fuzzy_out)
    mamdani_pred = predict_flexibility(mamdani_score)
    mamdani_preds.append(mamdani_pred)

    # Sugeno
    sugeno_score = defuzzify_sugeno(row['Age'], row['Device'], row['Financial Condition'], row['Internet Quality'])
    sugeno_pred = predict_flexibility(sugeno_score)
    sugeno_preds.append(sugeno_pred)

# === 9. Evaluasi Akurasi ===
true_labels = df['Flexibility_Level_Label'].tolist()
from sklearn.metrics import accuracy_score
mamdani_acc = accuracy_score(true_labels, mamdani_preds)
sugeno_acc = accuracy_score(true_labels, sugeno_preds)

print("Akurasi Mamdani:", round(mamdani_acc * 100, 2), "%")
print("Akurasi Sugeno:", round(sugeno_acc * 100, 2), "%")

# === 10. Output Prediksi Flexibility Level ===
df['Predicted_Mamdani'] = le_flex.inverse_transform(mamdani_preds)
df['Predicted_Sugeno'] = le_flex.inverse_transform(sugeno_preds)

# Tampilkan 10 hasil awal
print(df[['Age', 'Device', 'Financial Condition', 'Internet Quality', 'Flexibility Level', 'Predicted_Mamdani', 'Predicted_Sugeno']].head(10))


Education Level        0
Institution Type       0
Gender                 0
Age                    0
Device                 0
IT Student             0
Location               0
Financial Condition    0
Internet Type          0
Network Type           0
Flexibility Level      0
dtype: int64
Akurasi Mamdani: 39.83 %
Akurasi Sugeno: 47.14 %
   Age  Device  Financial Condition  Internet Quality Flexibility Level  \
0   23       2                    2                 2          Moderate   
1   23       1                    2                 2          Moderate   
2   18       1                    2                 2          Moderate   
3   11       1                    2                 2          Moderate   
4   18       1                    1                 2               Low   
5   11       1                    1                 2               Low   
6   11       1                    2                 2               Low   
7   11       1                    2                 2          